# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [44]:
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.window import Window

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd


spark = SparkSession.builder.appName("Spark SQL Quiz").getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

user_log.createOrReplaceTempView("user_log_table")
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

In [3]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id ""(empty string) NOT visit?

In [10]:
# TODO: write your code to answer question 1
spark.sql("SELECT DISTINCT page \
          FROM user_log_table \
          WHERE userId = ''").show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [11]:
spark.sql("SELECT DISTINCT page \
          FROM user_log_table").show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [12]:
spark.sql("SELECT * \
            FROM ( \
                SELECT DISTINCT page \
                FROM user_log_table \
                WHERE userID='') AS user_pages \
            RIGHT JOIN ( \
                SELECT DISTINCT page \
                FROM user_log_table) AS all_pages \
            ON user_pages.page = all_pages.page \
            WHERE user_pages.page IS NULL").show()

+----+----------------+
|page|            page|
+----+----------------+
|null|Submit Downgrade|
|null|       Downgrade|
|null|          Logout|
|null|   Save Settings|
|null|        Settings|
|null|        NextSong|
|null|         Upgrade|
|null|           Error|
|null|  Submit Upgrade|
+----+----------------+



In [16]:
spark.sql("SELECT * \
            FROM ( \
                SELECT DISTINCT page \
                FROM user_log_table \
                WHERE userID='') AS user_pages \
            RIGHT JOIN ( \
                SELECT DISTINCT page \
                FROM user_log_table) AS all_pages \
            ON user_pages.page = all_pages.page \
            WHERE user_pages.page IS NULL").show()

+----+----------------+
|page|            page|
+----+----------------+
|null|Submit Downgrade|
|null|       Downgrade|
|null|          Logout|
|null|   Save Settings|
|null|        Settings|
|null|        NextSong|
|null|         Upgrade|
|null|           Error|
|null|  Submit Upgrade|
+----+----------------+



In [21]:
spark.sql("SELECT * \
            FROM ( \
                SELECT DISTINCT page \
                FROM user_log_table \
                WHERE userID='') AS user_pages \
            RIGHT JOIN ( \
                SELECT DISTINCT page \
                FROM user_log_table) AS all_pages \
            ON user_pages.page = all_pages.page \
            WHERE user_pages.page IS NOT NULL").show()

+-----+-----+
| page| page|
+-----+-----+
| Home| Home|
|About|About|
|Login|Login|
| Help| Help|
+-----+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [25]:
# TODO: write your code to answer question 3
spark.sql("select count(distinct userId) from user_log_table where gender = 'F'").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [36]:
# TODO: write your code to answer question 4
spark.sql("select artist, count(artist) as plays from user_log_table group by artist order by plays DESC limit 1").show()

+--------+-----+
|  artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



In [38]:
play_counts = spark.sql("select artist, count(artist) as plays from user_log_table group by artist"); play_counts.show()

+--------------------+-----+
|              artist|plays|
+--------------------+-----+
|      The Black Keys|   40|
|        STRATOVARIUS|    1|
|      The Chameleons|    1|
|Dashboard Confess...|    3|
|      Jarabe De Palo|    3|
|        Ziggy Marley|    1|
|        Yann Tiersen|   10|
|  The Watts Prophets|    1|
|            Goldfish|    1|
|           Kate Nash|    3|
|              DJ Taz|    1|
|    Jane's Addiction|    1|
|         Eva Cassidy|    4|
|               Rufio|    1|
|           Los Lobos|    4|
|         Silverstein|    1|
|        Rhett Miller|    1|
|              Nebula|    1|
|Yonder Mountain S...|    1|
|        Generation X|    1|
+--------------------+-----+
only showing top 20 rows



In [39]:
play_counts.createOrReplaceTempView("artist_counts")

In [40]:
spark.sql("select max(plays) as max_plays from artist_counts").show()

+---------+
|max_plays|
+---------+
|       83|
+---------+



In [42]:
spark.sql("SELECT t2.artist, t2.plays \
           FROM (SELECT max(plays) AS max_plays \
                 FROM artist_counts) AS t1 \
           JOIN artist_counts AS t2 \
           ON t1.max_plays = t2.plays").show()

+--------+-----+
|  artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [46]:
# TODO: write your code to answer question 5
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql("SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home FROM user_log_table \
            WHERE (page = 'NextSong') or (page = 'Home') \
            ")

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql("SELECT *, SUM(is_home) OVER \
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_home_table")

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
spark.sql("SELECT AVG(count_results) FROM \
          (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

